# **Vue PIT : Point-in-Time Sale**


La vue PIT combine tous les éléments nécessaires à l’analyse fonctionnelle.

Elle assemble :

La clé de vente (Link)

Les clés métiers : product_id, region, date

Les attributs métier issus du satellite

In [36]:
%%sql 
CREATE TABLE  PIT_Sale AS
SELECT
    l.link_sale_hashkey,
    hp.product_id,
    hr.store_id,
    hd.date,
    s.units_sold,
    s.sales_revenue,
    s.promo_pct,
    s.marketing_spend,
    s.holiday_flag
FROM Link_Sale l
JOIN Hub_Product hp ON l.hub_product_hashkey = hp.hub_product_hashkey
JOIN Hub_store hr  ON l.hub_store_hashkey = hr.hub_store_hashkey
JOIN Hub_Date hd    ON l.hub_date_hashkey = hd.hub_date_hashkey
JOIN Sat_Sale s     ON l.link_sale_hashkey = s.link_sale_hashkey;


StatementMeta(, a224fc25-90d4-4a93-8ef1-8bcda58031bb, 37, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [35]:
%%sql
SELECT
    s.units_sold,
    s.sales_revenue,
    s.promo_pct,
    s.marketing_spend,
    s.holiday_flag
FROM Link_Sale l
JOIN Sat_Sale s     ON l.link_sale_hashkey = s.link_sale_hashkey;


StatementMeta(, a224fc25-90d4-4a93-8ef1-8bcda58031bb, 36, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 5 fields>

# **Création de la table fact pour Power BI**

In [38]:
%%sql
CREATE OR REPLACE TABLE fact_sales_prom
AS
SELECT
    date,
    store_id,
    product_id,
    CASE WHEN promo_pct > 0 THEN 1 ELSE 0 END AS promo_flag,
    SUM(units_sold) AS total_units,
    SUM(sales_revenue) AS total_sales,
    SUM(marketing_spend) AS total_marketing,
    SUM(sales_revenue) / NULLIF(SUM(marketing_spend), 0) AS ROI
FROM PIT_Sale
GROUP BY date, store_id, product_id, promo_pct;


StatementMeta(, a224fc25-90d4-4a93-8ef1-8bcda58031bb, 39, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [39]:
%%sql 
SELECT * FROM End_to_end_retail.fact_sales_promo

StatementMeta(, a224fc25-90d4-4a93-8ef1-8bcda58031bb, 40, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 8 fields>